In [1]:
# cd to the main directory
%cd ..

/home/tourloid/Desktop/PhD/Code/SPVD


# Create a base dataset to load raw data from PartNet

This initial version of PartNet is used to load the raw data in order to process them. This is not used as a training dataset.

In [2]:
#export
import h5py
import glob
import os
import torch
from torch.utils.data import Dataset
import numpy as np
import json
from datasets.utils import NoiseSchedulerDDPM
import random
from torchsparse.utils.quantize import sparse_quantize
from torchsparse import SparseTensor
from torch.utils.data import DataLoader
from torchsparse.utils.collate import sparse_collate_fn

categories = ['Faucet', 'Chair', 'Display', 'Knife', 'Table', 'Laptop', 'Refrigerator', 'Microwave', 
              'StorageFurniture', 'Bowl', 'Scissors', 'Door', 'TrashCan','Bed', 'Keyboard', 'Clock', 
              'Bottle', 'Bag', 'Lamp', 'Earphone', 'Vase', 'Dishwasher', 'Mug', 'Hat']

In [3]:
#export 
class PartNet(Dataset):
    """
        This dataset class loads data from .h5 and .json files and returns them unprocessed. 
    """
    def __init__(self, root_path, category, split='train', n_points=10000):
        super().__init__()

        all_categories = [f for f in os.listdir(root_path) if os.path.isdir(os.path.join(root_path, f))]
        assert category in all_categories, all_categories
        assert split in ['train', 'test', 'val'], split

        self.category = category
        self.split = split
        # create the path for the specific category
        category_path = os.path.join(root_path, category)
        
        # read all .h5 files and .json files
        self.points, self.labels, self.colors, self.meta = self.load_files(category_path, split)
        self.points = self.points[:, :n_points]
        self.labels = self.labels[:, :n_points]
        self.colors = self.colors[:, :n_points]

    def load_files(self, category_path, split):
        all_points, all_labels, all_colors = [], [], []
        all_meta = []
        file_pattern = f'{split}*.h5'
        file_paths = glob.glob(os.path.join(category_path, file_pattern))
        
        for file_path in file_paths:
            with h5py.File(file_path, 'r') as f:
                points = f['pts'][:]
                labels = f['label'][:]
                colors = f['rgb'][:]
                all_points.append(points)
                all_labels.append(labels)
                all_colors.append(colors)

            json_file = file_path.split('.')[0] + '.json'
            with open(json_file, 'r') as f:
                data = json.load(f)
                all_meta.extend(data)

        all_points = np.concatenate(all_points, axis=0)
        all_labels = np.concatenate(all_labels, axis=0)
        all_colors = np.concatenate(all_colors, axis=0)
        
        return all_points, all_labels, all_colors, all_meta

    def __len__(self):
        return len(self.points)

    def __getitem__(self, idx):

        pc, label, color, metadata = self.points[idx], self.labels[idx], self.colors[idx], self.meta[idx]
        return pc, label, color, metadata

In [4]:
path = '/home/tourloid/Downloads/PartNet/ins_seg_h5/ins_seg_h5/'
partnet = PartNet(path, 'Chair', 'train', n_points=2048)

# Process the dataset

We aim to create a version of the dataset where the subparts of objects have a substantial number of points, thereby excluding the very small parts.

## Represent the data as a tree structure
Note: This code needs refinement

In [21]:
class TreeNode:
    def __init__(self, name, parent=None):
        self.name = name
        self.parent = parent
        self.categories = set()
        self.children = {}
        self.label_count = 0  # To store frequency of the label in the point cloud

    def add_child(self, child, category=None, is_leaf=False):
        identifier = f"{child}_{len(self.children)}" if is_leaf else child
        if identifier not in self.children:
            new_node = TreeNode(child, self)
            if category is not None:
                if isinstance(category, list):  # Handle list of categories
                    new_node.categories.update(category)
                else:
                    new_node.categories.add(category)
            self.children[identifier] = new_node
        return self.children[identifier]

    def __repr__(self, level=0):
        category_str = ', '.join(map(str, self.categories)) if self.categories else 'None'
        ret = "\t" * level + repr(self.name) + f" (Categories: {category_str}, Count: {self.label_count})\n"
        for child in self.children.values():
            ret += child.__repr__(level + 1)
        return ret


class Tree:
    def __init__(self):
        self.root = TreeNode('root')

    def insert(self, path, category):
        parts = path.split('/')
        current_node = self.root
        for i, part in enumerate(parts):
            is_leaf = (i == len(parts) - 1)
            current_node = current_node.add_child(part, category if is_leaf else None, is_leaf)

    def update_node_label_count(self, category, count):
        def update(node):
            if category in node.categories:
                node.label_count += count
            for child in node.children.values():
                update(child)
        update(self.root)

    def merge_low_count_leaves(self, threshold):
        def collect_leaves(node, leaves):
            if node.children:
                for child in node.children.values():
                    collect_leaves(child, leaves)
            else:
                leaves.append(node)

        leaves = []
        collect_leaves(self.root, leaves)

        # Filter leaves below the threshold
        low_count_leaves = [leaf for leaf in leaves if leaf.label_count < threshold]
        name_to_leaves = {}
        for leaf in low_count_leaves:
            name_to_leaves.setdefault(leaf.name.split('_')[0], []).append(leaf)

        # Merge leaves with the same base name
        for name, leaves in name_to_leaves.items():
            if len(leaves) > 1:
                # Accumulate counts and categories
                target_leaf = leaves[0]
                for leaf in leaves[1:]:
                    target_leaf.label_count += leaf.label_count
                    target_leaf.categories.update(leaf.categories)
                    # Remove leaf from parent
                    if leaf.parent:
                        keys_to_remove = [key for key, val in leaf.parent.children.items() if val == leaf]
                        for key in keys_to_remove:
                            del leaf.parent.children[key]

    def get_leaf_nodes(self):
        leaves = []
        def collect_leaves(node):
            if not node.children:  # If no children, it's a leaf node
                leaves.append(node)
            else:
                for child in node.children.values():
                    collect_leaves(child)
        collect_leaves(self.root)
        return leaves
    
    def __repr__(self):
        return self.root.__repr__()

    def display_tree(self):
        print(self.__repr__())

def build_tree(node_info):
    tree = Tree()
    for path, category in node_info:
        tree.insert(path, category)
    return tree


In [28]:
def process_dataset(partnet, threshold=300):
    all_pc = []
    all_labels = []
    all_colors = []
    
    
    for data in partnet:
    
        pc, labels, colors, metadata = data
        part_meta = metadata['ins_seg']
    
        part_names = []
        part_ids = []
    
        for item in part_meta:
            part_name = item['part_name']
            part_names.append(part_name)
    
            leaf_id = item['leaf_id_list']
            part_ids.append(leaf_id)
    
        node_info = [(node_name, part_id) for node_name, part_id in zip(part_names, part_ids)]
    
        filtered_node_info = []
        for ni in node_info:
            if len(ni[1]) > 1: continue
            filtered_node_info.append(ni)
        
        # use the filtered node info to create a tree structure of the point cloud
        tree = build_tree(filtered_node_info)
        
        # update the number of points in each label
        unique, counts = np.unique(labels, return_counts=True)
        label_frequencies = dict(zip(unique, counts))
    
        for label, count in label_frequencies.items():
            tree.update_node_label_count(label, count)


        # merge nodes with a low number of points
        tree.merge_low_count_leaves(threshold)

        leaf_nodes = tree.get_leaf_nodes()
        
        for node in leaf_nodes: 
            cats = node.categories
            if len(cats) > 1:
                main_cat = cats.pop()
                for c in cats:
                    inds = labels == c
                    labels[inds] = main_cat

        num_parts = len(np.unique(labels))
        
        all_pc.append(pc)
        all_labels.append(labels)
        all_colors.append(colors)
    
    all_pc = np.stack(all_pc)
    all_labels = np.stack(all_labels)
    all_colors = np.stack(all_colors)

    return all_pc, all_labels, all_colors

## Creation of the dataset - Preprocessing the data
To generate the data run the following function: See example bellow.

In [29]:
def create_data(data_path, categories, save_path='./data/PartNetProcessed/', n_points=2048, threshold=40):
    splits = ['train', 'test', 'val']
    for category in categories:
        for split in splits:
            dataset = PartNet(path, category, split, n_points)
            all_pc, all_labels, _ = process_dataset(dataset, threshold=threshold)
            np.save(os.path.join(save_path, f'{category}_{split}_points.npy'), all_pc)
            np.save(os.path.join(save_path, f'{category}_{split}_labels.npy'), all_labels)

In [30]:
create_data(path, categories = ['Chair', 'Table'], threshold=40)

## Create a Dataset class to load the generated data

In [5]:
#export
class PartNetCompletion(Dataset):

    def __init__(self, root_path, category, split='train', n_points=2048):
        super().__init__()

        assert split in ['train', 'test', 'val'], split

        # path to load points and labels
        point_path = os.path.join(root_path, f'{category}_{split}_points.npy')
        label_path = os.path.join(root_path, f'{category}_{split}_labels.npy')

        # load points and labels
        points = np.load(point_path)
        labels = np.load(label_path)

        # keep n_points
        self.points = points[:, :n_points]
        self.labels = labels[:, :n_points]

    def __len__(self):
        return len(self.points)

    def __getitem__(self, idx):
        pc, labels = self.points[idx], self.labels[idx]  
        return pc, labels

### Visualize Data

In [6]:
from utils.visualization import vis_pc_sphere

def generate_colors(num_labels):
    # Generates a unique color for each label
    return np.random.rand(num_labels, 3)  # Random RGB colors

def label_to_color_tensor(point_labels):
    # Flatten to handle in a single array (assumes point_labels is a 1D array of labels for simplicity)
    unique_labels = np.unique(point_labels)
    num_labels = len(unique_labels)
    
    # Generate colors
    colors = generate_colors(num_labels)
    
    # Create a dictionary mapping label to color
    label_to_color_map = {label: colors[i] for i, label in enumerate(unique_labels)}
    
    # Map each point's label to its color
    color_tensor = np.array([label_to_color_map[label] for label in point_labels])
    
    return color_tensor

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
dataset = PartNetCompletion('./data/PartNetProcessed/', 'Chair')

In [8]:
points, labels = dataset[0]
color = label_to_color_tensor(labels)
vis_pc_sphere(torch.tensor(points), radius=0.05, color=torch.tensor(color))

### Dataset for Completion

In [32]:
#export 
class PartNetCompletionNoisySparse(PartNetCompletion):

    def __init__(self, root_path, category, split='train', n_points=2048, pres=1e-5, min_num_parts=3, add_real_info=True,
                  ddpm_params = {'beta_min':0.0001, 
                                 'beta_max':0.02, 
                                 'n_steps' :1000, 
                                 'mode'    :'linear'}):
        super().__init__(root_path, category, split, n_points)
        self.pres = pres
        self.noise_scheduler = NoiseSchedulerDDPM(beta_min=ddpm_params['beta_min'],
                                                  beta_max=ddpm_params['beta_max'],
                                                  n_steps =ddpm_params['n_steps'],
                                                  mode    =ddpm_params['mode'])

        self.min_num_parts = min_num_parts
        self.add_real_info = add_real_info
        
    def __getitem__(self, idx):
        points, labels = super().__getitem__(idx)

        # normalize point cloud (mean=0, std=1)
        points = points - points.mean(axis=0)
        points = points / points.std()
        
        # select parts to discard
        unique_labels = np.unique(labels)
        num_parts = len(unique_labels)
        if num_parts > self.min_num_parts:
            keep_parts = random.randint(self.min_num_parts, num_parts - 1)
        else:
            keep_parts = random.randint(1, num_parts-1)
            
        keep_labels = np.random.choice(unique_labels, keep_parts, replace=False)
        
        # create a mask        
        labels = torch.tensor(labels)
        keep_labels = torch.tensor(keep_labels)
        mask = torch.isin(labels, keep_labels)

        # add noise
        points = torch.tensor(points)
        noisy_pc, t, noise = self.noise_scheduler(points)

        # keep the original points for the masked areas
        noisy_pc[mask] = points[mask]
                
        # voxelize
        coords = noisy_pc.numpy()
        coords = coords - np.min(coords, axis=0, keepdims=True)
        coords, indices = sparse_quantize(coords, self.pres, return_index=True)

        coords = torch.tensor(coords)

        if self.add_real_info:
            noisy_pc = torch.cat([noisy_pc, mask.unsqueeze(-1).float()], dim=-1)
        
        feats = noisy_pc[indices]
        noise = noise[indices]
        mask  = mask[indices]
        
        noisy_pc = SparseTensor(coords=coords, feats=feats)
        
        return {
            'input':noisy_pc,
            't': t,
            'noise': noise,
            'mask': mask
        }

In [33]:
dataset = PartNetCompletionNoisySparse('./data/PartNetProcessed/', 'Chair')

In [37]:
res = dataset[0]
pc, mask = res['input'], res['mask']

In [38]:
vis_pc_sphere(pc.F[mask, :3], radius=0.05)

In [39]:
#export 
def get_sparse_completion_datasets(path, category, n_points=2048, pres=1e-5, min_num_parts=3, add_real_info=True,
                                    ddpm_params = {'beta_min':0.0001, 
                                                   'beta_max':0.02, 
                                                   'n_steps':1000, 
                                                   'mode':'linear' }):

    tr_dataset = PartNetCompletionNoisySparse(path, category, split='train', n_points=n_points, pres=pres, min_num_parts=min_num_parts, 
                                                  add_real_info=add_real_info, ddpm_params=ddpm_params) 
    te_dataset = PartNetCompletionNoisySparse(path, category, split='test', n_points=n_points, pres=pres, min_num_parts=min_num_parts, 
                                                  add_real_info=add_real_info, ddpm_params=ddpm_params)
    
    return tr_dataset, te_dataset

In [40]:
#export 
def get_sparse_completion_dataloaders(path, category, n_points=2048, pres=1e-5, min_num_parts=3, add_real_info=True, batch_size=32, num_workers=8,
                                       ddpm_params = {'beta_min':0.0001, 
                                                      'beta_max':0.02, 
                                                      'n_steps':1000, 
                                                      'mode':'linear'}):
    tr_dataset, te_dataset = get_sparse_completion_datasets(path, category=category, n_points=n_points, pres=pres, min_num_parts=min_num_parts, 
                                                            add_real_info=add_real_info, ddpm_params=ddpm_params)

    tr_dl = DataLoader(tr_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True, collate_fn=sparse_collate_fn)
    te_dl = DataLoader(te_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=sparse_collate_fn)

    return tr_dl, te_dl

In [41]:
tr_dl, te_dl = get_sparse_completion_dataloaders('./data/PartNetProcessed/', 'Chair')
len(tr_dl), len(te_dl)

(140, 39)

In [42]:
len(tr_dl.dataset), len(te_dl.dataset)

(4489, 1217)

In [43]:
for batch in tr_dl:
    pc = batch['input']

# Generation of Colored Point Cloud Data
- Use farthest point sample to sample the points -- later note: fps is not required...

In [ ]:
import open3d as o3d
from tqdm import tqdm

In [ ]:
path = '/home/tourloid/Downloads/PartNet/ins_seg_h5/ins_seg_h5/'
categories = ['Chair']
splits = ['train', 'test', 'val']

In [ ]:
def process_dataset(dataset, save_path = './data/ShapeNetColor', s_points=10000):
    category, split = dataset.category, dataset.split
    print(f' Processing PartNet category : {category} | split: {split}')
    all_pcs = []
    all_clr = []
    for data in dataset:
        pc, label, color, metadata = data
        color = color.astype(np.float32) / 255.
        
        # create open3d point cloud
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(pc)
        pcd.colors = o3d.utility.Vector3dVector(color)
        
        # fps 
        pcd_fps = pcd.farthest_point_down_sample(s_points)
        
        fps_points = np.asarray(pcd_fps.points)
        fps_colors = np.asarray(pcd_fps.colors)
    
        all_pcs.append(fps_points)
        all_clr.append(fps_colors)
    
    all_pcs = np.stack(all_pcs)
    all_clr = np.stack(all_clr)

    all_clr_pcs = np.concatenate([all_pcs, all_clr], axis=-1)

    name = f'{category}_{split}.npy'
    np.save(os.path.join(save_path, name), all_clr_pcs)

In [ ]:
for category in categories:
    for split in splits:
        partnet = PartNet(path, category, split)
        process_dataset(partnet)

## Create a dataset that will load these processed pointclouds

In [ ]:
#export 
class ShapeNetColor(Dataset):
    def __init__(self, root_path, category, split='train', n_points=2048, max_points=10000):
        '''
        Processing: 
            1. Normalize color values globally (dataset provides inverse function to retrieve correct color values)
            2. Normalize point coordinates per shape ==> mean=0, std=1
        '''
        super().__init__()

        self.category, self.split, self.n_points, self.max_points = category, split, n_points, max_points
        
        data_path = os.path.join(root_path, f'{category}_{split}.npy')
        data = np.load(data_path)
        data = data[:, :max_points, :]

        # normalize color values
        self.m = data[..., 3:].mean(axis=1).mean(axis=0)
        data[..., 3:] = data[..., 3:] - self.m
        self.s = data[..., 3:].std()
        data[..., 3:] = data[..., 3:] / self.s

        self.data = data

    def get_actual_color(self, clr):
        return (clr * self.s + self.m).clamp(0, 1)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]

        if self.max_points > self.n_points:
            # select a random sabsample of the points
            indices = np.arange(self.n_points)
            np.random.shuffle(indices)
            data = data[indices]

        # normalize coordinates
        data[:, :3] = data[:, :3] - data[:, :3].mean(axis=0)
        data[:, :3] = data[:, :3] / data[:, :3].std()

        return torch.tensor(data)

In [ ]:
shapenetcolour = ShapeNetColor('./data/ShapeNetColor/', category='Chair', max_points=4096)

In [ ]:
shapenetcolour[0].mean(axis=0), shapenetcolour[0].std()

In [ ]:
#export

class ShapeNetColorSparseNoisy(ShapeNetColor):

    def __init__(self, root_path, category, split='train', n_points=2048, max_points=10000, 
                 pres=1e-5, ddpm_params = {'beta_min':0.0001, 
                                           'beta_max':0.02, 
                                           'n_steps':1000, 
                                           'mode':'linear' }):
        super().__init__(root_path, category, split, n_points, max_points)
        self.pres = pres
       
        self.noise_scheduler = NoiseSchedulerDDPM(ddpm_params['beta_min'], ddpm_params['beta_max'], ddpm_params['n_steps'], ddpm_params['mode'])

    def __getitem__(self, idx):

        data = super().__getitem__(idx)

        # add noise
        data, t, noise = self.noise_scheduler(data)

        # sparse quantize
        pts = data[:, :3].numpy()
        coords = pts - np.min(pts, axis=0, keepdims=True)
        coords, indices = sparse_quantize(coords, self.pres, return_index=True)

        # coordinates as torch tensor
        coords = torch.tensor(coords, dtype=torch.int)
        # features (includes actual coordinates and colors)
        feats = data[indices].float()
        noise = noise[indices].float()
        
        noisy_pts = SparseTensor(coords=coords, feats=feats)
        noise = SparseTensor(coords=coords, feats=noise)
        
        return {'input':noisy_pts, 't':t, 'noise':noise}

In [ ]:
shapenetcolour = ShapeNetColorSparseNoisy('./data/ShapeNetColor/', category='Chair', max_points=4096)

In [ ]:
#export 
def get_sparse_datasets(path, category, pres=1e-5, n_points=2048, max_points=4096,
                        ddpm_params = {'beta_min':0.0001, 
                                       'beta_max':0.02, 
                                       'n_steps':1000, 
                                       'mode':'linear' }):

    tr_dataset = ShapeNetColorSparseNoisy(path, category, split='train', n_points=n_points, max_points=max_points, pres=pres, ddpm_params=ddpm_params) 
    te_dataset = ShapeNetColorSparseNoisy(path, category, split='test' , n_points=n_points, max_points=max_points, pres=pres, ddpm_params=ddpm_params)

    return tr_dataset, te_dataset

In [ ]:
#export 
def get_sparse_dataloaders(path, category, pres=1e-5, n_points=2048, max_points=4096, batch_size=32, num_workers=8,
                           ddpm_params = {'beta_min':0.0001, 
                                          'beta_max':0.02, 
                                          'n_steps':1000, 
                                          'mode':'linear'}):
    tr_dataset, te_dataset = get_sparse_datasets(path, category=category, pres=pres, n_points=n_points, max_points=max_points, ddpm_params=ddpm_params)

    tr_dl = DataLoader(tr_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True, collate_fn=sparse_collate_fn)
    te_dl = DataLoader(te_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=sparse_collate_fn)

    return tr_dl, te_dl

In [ ]:
tr_dl, te_dl = get_sparse_dataloaders('./data/ShapeNetColor/', 'Chair')

In [ ]:
len(tr_dl), len(te_dl)